```
Copyright 2024 Nikolai Körber. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# EGIC (TF 2)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Install Dependencies

In [ ]:
!pip install immutabledict
!pip install tensorflow-datasets
!pip install tensorflow-compression~=$(pip show tensorflow | perl -p -0777 -e 's/.*Version: (\d+\.\d+).*/\1.0/sg')
!pip install protobuf==3.20.0
!pip install "numpy<1.24"

## Test that TFC works correctly

In [ ]:
# If you encounter any issues, see https://github.com/tensorflow/compression (Installation/ Colab)
!python -m tensorflow_compression.all_tests

## TensorBoard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir /tf/notebooks/EGIC/res/train_kkshms2024 --host 0.0.0.0 --port=6011

## Start Training

### Stage 1 (warm-up)

In [ ]:
# caution: parsing arguments might be ignored for kkshms2024 (e.g. model definition)
!python /tf/notebooks/EGIC/src/kkshms2024.py -V --model_path /tf/notebooks/EGIC/res/kkshms2024 train --max_support_slices 10 --warm_up 1 --lambda 2 --lr 1e-4 --epochs 200 --steps_per_epoch 10000 --train_path /tf/notebooks/EGIC/res/train_kkshms2024

### Stage 2 (full learning objective)

In [ ]:
# caution: parsing arguments might be ignored for kkshms2024 (e.g. model definition)
!python /tf/notebooks/EGIC/src/kkshms2024.py -V --model_path /tf/notebooks/EGIC/res/kkshms2024 train --max_support_slices 10 --warm_up 0 --lambda 2 --lr 1e-5 --epochs 300 --steps_per_epoch 10000 --init_disc 1 --train_path /tf/notebooks/EGIC/res/train_kkshms2024

## Compression


In [ ]:
# evaluate single image
!python /tf/notebooks/EGIC/src/kkshms2024.py -V --model_path /tf/notebooks/EGIC/res/kkshms2024 compress /tf/notebooks/EGIC/res/data/kodak/kodim19.png /tf/notebooks/EGIC/res/eval/kodim19.tfci

In [ ]:
# compress only
!python /tf/notebooks/EGIC/src/kkshms2024.py --model_path /tf/notebooks/EGIC/res/kkshms2024 compress /tf/notebooks/EGIC/res/data/kodak/kodim19.png /tf/notebooks/EGIC/res/eval/kodim19.tfci

In [ ]:
# decompress only
!python /tf/notebooks/EGIC/src/kkshms2024.py --model_path /tf/notebooks/EGIC/res/kkshms2024 decompress /tf/notebooks/EGIC/res/eval/kodim19.tfci /tf/notebooks/EGIC/res/eval/kodim19_hat.png

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.rcParams["figure.figsize"] = (16,8)

x = mpimg.imread('/tf/notebooks/EGIC/res/data/kodak/kodim19.png')
x_hat = mpimg.imread('/tf/notebooks/EGIC/res/eval/kodim19_hat.png')

f = plt.figure()
f.add_subplot(1,2, 1)
plt.imshow(x)
plt.title('x')
f.add_subplot(1,2, 2)
plt.imshow(x_hat)
plt.title('x_hat')
plt.show()

## Evaluate Kodak

In [ ]:
# only bpp, PSNR -> we compute FID and KID following https://github.com/facebookresearch/NeuralCompression/blob/main/projects/illm/eval_folder_example.py
!python /tf/notebooks/EGIC/src/kkshms2024.py -V --model_path /tf/notebooks/EGIC/res/kkshms2024 evaluate_ds --images_glob=../res/data/kodak/*.png --out_dir=../res/eval/SwinT-ChARM-Perceptual/kodak/

## Evaluate DIV2K

In [ ]:
# only bpp, PSNR -> we compute FID and KID following https://github.com/facebookresearch/NeuralCompression/blob/main/projects/illm/eval_folder_example.py
!python /tf/notebooks/EGIC/src/kkshms2024.py -V --model_path /tf/notebooks/EGIC/res/kkshms2024 evaluate_ds --images_glob=../res/data/DIV2K_valid_HR/*.png --out_dir=../res/eval/SwinT-ChARM-Perceptual/DIV2K_valid_HR/

## Evaluate CLIC 2020

In [ ]:
# only bpp, PSNR -> we compute FID and KID following https://github.com/facebookresearch/NeuralCompression/blob/main/projects/illm/eval_folder_example.py
!python /tf/notebooks/EGIC/src/kkshms2024.py -V --model_path /tf/notebooks/EGIC/res/kkshms2024 evaluate_ds --images_glob=../res/data/clic2020/*/*.png --out_dir=../res/eval/SwinT-ChARM-Perceptual/clic2020/